In [2]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
import pyspark.sql.functions as F

# Create a SparkConf object
conf = SparkConf().setAppName("learning")\
            .setMaster("local[4]")\

spark = SparkSession.builder.config(conf=conf).getOrCreate()
spark

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/07/25 01:20:35 WARN Utils: Your hostname, de24, resolves to a loopback address: 127.0.1.1; using 192.168.0.102 instead (on interface enp0s3)
25/07/25 01:20:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/25 01:20:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
'''
Table: Activity

+----------------+---------+
| Column Name    | Type    |
+----------------+---------+
| machine_id     | int     |
| process_id     | int     |
| activity_type  | enum    |
| timestamp      | float   |
+----------------+---------+
The table shows the user activities for a factory website.
(machine_id, process_id, activity_type) is the primary key (combination of columns with unique values) of 
this table.
machine_id is the ID of a machine.
process_id is the ID of a process running on the machine with ID machine_id.
activity_type is an ENUM (category) of type ('start', 'end').
timestamp is a float representing the current time in seconds.
'start' means the machine starts the process at the given timestamp and 'end' means the machine ends the process 
at the given 
timestamp.
The 'start' timestamp will always be before the 'end' timestamp for every (machine_id, process_id) pair.
 

There is a factory website that has several machines each running the same number of processes. Write a solution 
to find the 
average time each machine takes to complete a process.

The time to complete a process is the 'end' timestamp minus the 'start' timestamp. The average time is calculated 
by the total 
time to complete every process on the machine divided by the number of processes that were run.

The resulting table should have the machine_id along with the average time as processing_time, which should 
be rounded to 3 
decimal places.

Return the result table in any order.

The result format is in the following example.

 

Example 1:

Input: 
Activity table:
+------------+------------+---------------+-----------+
| machine_id | process_id | activity_type | timestamp |
+------------+------------+---------------+-----------+
| 0          | 0          | start         | 0.712     |
| 0          | 0          | end           | 1.520     |
| 0          | 1          | start         | 3.140     |
| 0          | 1          | end           | 4.120     |
| 1          | 0          | start         | 0.550     |
| 1          | 0          | end           | 1.550     |
| 1          | 1          | start         | 0.430     |
| 1          | 1          | end           | 1.420     |
| 2          | 0          | start         | 4.100     |
| 2          | 0          | end           | 4.512     |
| 2          | 1          | start         | 2.500     |
| 2          | 1          | end           | 5.000     |
+------------+------------+---------------+-----------+
Output: 
+------------+-----------------+
| machine_id | processing_time |
+------------+-----------------+
| 0          | 0.894           |
| 1          | 0.995           |
| 2          | 1.456           |
+------------+-----------------+
Explanation: 
There are 3 machines running 2 processes each.
Machine 0's average time is ((1.520 - 0.712) + (4.120 - 3.140)) / 2 = 0.894
Machine 1's average time is ((1.550 - 0.550) + (1.420 - 0.430)) / 2 = 0.995
Machine 2's average time is ((4.512 - 4.100) + (5.000 - 2.500)) / 2 = 1.456
'''

In [3]:
data = [
(0,0,'start',0.712),
(0,0,'end'  ,1.520),
(0,1,'start',3.140),
(0,1,'end'  ,4.120),
(1,0,'start',0.550),
(1,0,'end'  ,1.550),
(1,1,'start',0.430),
(1,1,'end'  ,1.420),
(2,0,'start',4.100),
(2,0,'end'  ,4.512),
(2,1,'start',2.500),
(2,1,'end'  ,5.000)
]
schema = ['machine_id','process_id','activity_type','timestamp']

In [5]:
df = spark.createDataFrame(data = data, schema = schema)
df.show()

+----------+----------+-------------+---------+
|machine_id|process_id|activity_type|timestamp|
+----------+----------+-------------+---------+
|         0|         0|        start|    0.712|
|         0|         0|          end|     1.52|
|         0|         1|        start|     3.14|
|         0|         1|          end|     4.12|
|         1|         0|        start|     0.55|
|         1|         0|          end|     1.55|
|         1|         1|        start|     0.43|
|         1|         1|          end|     1.42|
|         2|         0|        start|      4.1|
|         2|         0|          end|    4.512|
|         2|         1|        start|      2.5|
|         2|         1|          end|      5.0|
+----------+----------+-------------+---------+



In [13]:
df_tmp = df.groupBy(F.col("machine_id"),F.col("process_id"))\
            .agg((F.max(F.col("timestamp")) - F.min(F.col("timestamp"))).alias("difference"))

df_tmp.show()

df_tmp.groupBy(F.col("machine_id"))\
      .agg(F.round(F.avg(F.col("difference")),3).alias("processing_time"))\
      .show(truncate=False)
    

+----------+----------+------------------+
|machine_id|process_id|        difference|
+----------+----------+------------------+
|         0|         1|              0.98|
|         0|         0|             0.808|
|         1|         0|               1.0|
|         1|         1|              0.99|
|         2|         0|0.4119999999999999|
|         2|         1|               2.5|
+----------+----------+------------------+

+----------+---------------+
|machine_id|processing_time|
+----------+---------------+
|0         |0.894          |
|1         |0.995          |
|2         |1.456          |
+----------+---------------+



In [18]:
# 3 same as above query but combining
df.groupBy(F.col("machine_id"),F.col("process_id"))\
            .agg((F.max(F.col("timestamp")) - F.min(F.col("timestamp"))).alias("difference"))\
.groupBy(F.col("machine_id"))\
      .agg(F.round(F.avg(F.col("difference")),3).alias("processing_time"))\
      .show(truncate=False)

+----------+---------------+
|machine_id|processing_time|
+----------+---------------+
|0         |0.894          |
|1         |0.995          |
|2         |1.456          |
+----------+---------------+



## Anothor Approach

In [17]:
df.alias("s").join(df.alias("e"), 
                   (F.col("s.machine_id") == F.col("e.machine_id")) & (F.col("s.process_id") == F.col("e.process_id")),
                  'left')\
             .where((F.col("s.activity_type") == 'start') & (F.col("e.activity_type") == 'end'))\
             .groupBy(F.col("s.machine_id"))\
             .agg(F.round(F.avg(F.col("e.timestamp")-F.col("s.timestamp")),3).alias("processing_time"))\
             .show(truncate=False)

+----------+---------------+
|machine_id|processing_time|
+----------+---------------+
|0         |0.894          |
|1         |0.995          |
|2         |1.456          |
+----------+---------------+



## SQL Solution

<pre>
    WITH TEMP AS (
        SELECT machine_id, process_id, MAX(timestamp)-MIN(timestamp) as difference
        FROM Activity 
        GROUP BY machine_id, process_id
    )
    SELECT machine_id, AVG(difference)
    FROM TEMP
    GROUP BY TEMP;
</pre>

## Anothor Approach

<pre>
SELECT s.machine_id, ROUND(AVG(e.timestamp-s.timestamp), 3) AS processing_time
FROM Activity s JOIN Activity e ON s.machine_id = e.machine_id 
                AND s.process_id = e.process_id 
                AND s.activity_type = 'start' 
                AND e.activity_type = 'end'
    GROUP BY s.machine_id;
</pre>